In [54]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
from sklearn.model_selection import train_test_split

from tensorflow.keras.layers import Dense , Input, GlobalMaxPool1D, LSTM,Embedding
from tensorflow.keras.optimizers import Adam,SGD
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model

In [55]:
train = pd.read_csv('/Users/akashzamnani/datasets/NLP/twitter-nlp/twitter_training.csv')
train.head()

,2401,Borderlands,Positive,"im getting on borderlands and i will murder you all ,"
0,2401,Borderlands,Positive,I am coming to the borders and I will kill you...
1,2401,Borderlands,Positive,im getting on borderlands and i will kill you ...
2,2401,Borderlands,Positive,im coming on borderlands and i will murder you...
3,2401,Borderlands,Positive,im getting on borderlands 2 and i will murder ...
4,2401,Borderlands,Positive,im getting into borderlands and i can murder y...


In [56]:
test = pd.read_csv('/Users/akashzamnani/datasets/NLP/twitter-nlp/twitter_validation.csv')
test.head()

,3364,Facebook,Irrelevant,"I mentioned on Facebook that I was struggling for motivation to go for a run the other day, which has been translated by Tom’s great auntie as ‘Hayley can’t get out of bed’ and told to his grandma, who now thinks I’m a lazy, terrible person 🤣"
0,352,Amazon,Neutral,BBC News - Amazon boss Jeff Bezos rejects clai...
1,8312,Microsoft,Negative,@Microsoft Why do I pay for WORD when it funct...
2,4371,CS-GO,Negative,"CSGO matchmaking is so full of closet hacking,..."
3,4433,Google,Neutral,Now the President is slapping Americans in the...
4,6273,FIFA,Negative,Hi @EAHelp I’ve had Madeleine McCann in my cel...


In [57]:
train['Borderlands'].unique()

array(['Borderlands', 'CallOfDutyBlackopsColdWar', 'Amazon', 'Overwatch',
       'Xbox(Xseries)', 'NBA2K', 'Dota2', 'PlayStation5(PS5)',
       'WorldOfCraft', 'CS-GO', 'Google', 'AssassinsCreed', 'ApexLegends',
       'LeagueOfLegends', 'Fortnite', 'Microsoft', 'Hearthstone',
       'Battlefield', 'PlayerUnknownsBattlegrounds(PUBG)', 'Verizon',
       'HomeDepot', 'FIFA', 'RedDeadRedemption(RDR)', 'CallOfDuty',
       'TomClancysRainbowSix', 'Facebook', 'GrandTheftAuto(GTA)',
       'MaddenNFL', 'johnson&johnson', 'Cyberpunk2077',
       'TomClancysGhostRecon', 'Nvidia'], dtype=object)

In [58]:
train = train.dropna()
test = test.dropna()

In [59]:
test.isna().sum()

3364                                                                                                                                                                                                                                                  0
Facebook                                                                                                                                                                                                                                              0
Irrelevant                                                                                                                                                                                                                                            0
I mentioned on Facebook that I was struggling for motivation to go for a run the other day, which has been translated by Tom’s great auntie as ‘Hayley can’t get out of bed’ and told to his grandma, who now thinks I’m a lazy, terrible person 🤣    0
dtype: i

In [60]:
train['2401'].unique()

array([2401, 2402, 2403, ..., 9198, 9199, 9200])

In [61]:
train = train.drop(['2401'],axis=1)
train.columns = ['name','type','comment']
train = train.sample(n=len(train))
train = train.reset_index(drop=True)
train.head()

,name,type,comment
0,Battlefield,Irrelevant,A ban for Battlefield 4 player yoshistyles has...
1,HomeDepot,Neutral,23 A car rolled out of entering a church parki...
2,AssassinsCreed,Positive,WOW FUCK YOURE DEFENDING THOSE COMMENTS. i thi...
3,CS-GO,Positive,Really want to get back into her senior squad ...
4,PlayerUnknownsBattlegrounds(PUBG),Irrelevant,It's only 2⃣ days before the PSL Elisa Viihde ...


In [62]:
test = test.drop(['3364'],axis=1)
test.columns = ['name','type','comment']
test = test.sample(n=len(test))
test = test.reset_index(drop=True)
test.head()

,name,type,comment
0,Facebook,Irrelevant,another reason why QT is superior 😌
1,Borderlands,Neutral,Borderlands® 3\nFirebug (Bronze)\nSabotage all...
2,WorldOfCraft,Neutral,zysola.blogspot.com/p/welcome.html… #zysola #t...
3,CallOfDuty,Irrelevant,Call of duty warzone (livestream) w/ subs #War...
4,AssassinsCreed,Positive,FUCKKKKKK I CANT WAIT


In [63]:
train['type'] = train['type'].map({'Irrelevant':0,'Negative':1,'Neutral':2,'Positive':3})

In [64]:
test['type'] = test['type'].map({'Irrelevant':0,'Negative':1,'Neutral':2,'Positive':3})

In [65]:
y_train = train['type'].values
y_test = test['type'].values
x_train = train['comment'].values
x_test = test['comment'].values

In [66]:
x_train

array(['A ban for Battlefield 4 player yoshistyles has occurred SEE DETAILS: bf4db.com/player/ban/170…',
       '23 A car rolled out of entering a church parking light lot and slid over from an embankment at a Hills Home Insurance Depot in El Dorado last week, crew and police believe a dog is hard to blame : arkansasonline. com / news / 2020 / jan / …',
       'WOW FUCK YOURE DEFENDING THOSE COMMENTS. i think about Bitcoin, but i got Assassins Creed on Blu Ray and they blew cigarette',
       ...,
       'I remember back before when people in Earth Battlefield 3 would put C4s stamped on on the military buggies..... 20 People still do this wtf',
       'Come watch me working to complete the!royalechallenge',
       'When I complete the battle pass and the officer rank progression challenges each season,  '],
      dtype=object)

In [68]:
MAX_VOCAB_SIZE = 20000
tokenizer = Tokenizer(MAX_VOCAB_SIZE)
tokenizer.fit_on_texts(x_train)
sequences_train = tokenizer.texts_to_sequences(x_train)
sequences_test = tokenizer.texts_to_sequences(x_test)

In [70]:
word2idx = tokenizer.word_index
N = len(word2idx)

In [72]:
x_train = pad_sequences(sequences_train)
x_train.shape

(73995, 166)

In [73]:
T = x_train.shape[1]

In [74]:
x_test = pad_sequences(sequences_test, maxlen=T)

In [75]:
D = 30

i = Input(shape=(T,))
x = Embedding(N+1,D)(i)
x = LSTM(25,activation='relu',return_sequences=True)(x)
x = GlobalMaxPool1D()(x)
x = Dense(4, activation='softmax')(x)

model = Model(i,x)

In [76]:
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [77]:
model.fit(x_train,y_train,epochs=20,validation_data=(x_test,y_test))

Epoch 1/20
2313/2313 ━━━━━━━━━━━━━━━━━━━━ 52s 22ms/step - accuracy: 0.4477 - loss: 18.7734 - val_accuracy: 0.6266 - val_loss: 0.9536
Epoch 2/20
2313/2313 ━━━━━━━━━━━━━━━━━━━━ 52s 22ms/step - accuracy: 0.6318 - loss: 0.9320 - val_accuracy: 0.7718 - val_loss: 0.6472
Epoch 3/20
2313/2313 ━━━━━━━━━━━━━━━━━━━━ 51s 22ms/step - accuracy: 0.7412 - loss: 0.8031 - val_accuracy: 0.8599 - val_loss: 0.4761
Epoch 4/20
2313/2313 ━━━━━━━━━━━━━━━━━━━━ 51s 22ms/step - accuracy: 0.8115 - loss: 0.5522 - val_accuracy: 0.8909 - val_loss: 0.3666
Epoch 5/20
2313/2313 ━━━━━━━━━━━━━━━━━━━━ 51s 22ms/step - accuracy: 0.8529 - loss: 0.4308 - val_accuracy: 0.9219 - val_loss: 0.2774
Epoch 6/20
2313/2313 ━━━━━━━━━━━━━━━━━━━━ 51s 22ms/step - accuracy: 0.8781 - loss: 0.3402 - val_accuracy: 0.9259 - val_loss: 0.2362
Epoch 7/20
2313/2313 ━━━━━━━━━━━━━━━━━━━━ 51s 22ms/step - accuracy: 0.8992 - loss: 0.2793 - val_accuracy: 0.9389 - val_loss: 0.2150
Epoch 8/20
2313/2313 ━━━━━━━━━━━━━━━━━━━━ 52s 22ms/step - accuracy: 0.9140 